# Libraries

In [1]:
# using conda library 'ontogpt_fork'; naming irrelevant as this was previously going to be used to work on a different project.

# increase cell-width
from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import re
import numpy as np
import os
import glob
from tqdm import tqdm # progress bar tracking

# API keys:
from dotenv import load_dotenv
load_dotenv()

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

import json
import jsonschema
from jsonschema import validate

# prompts for OpenAI

import openai_secret_manager
import openai
import time
import random

import tiktoken
from typing import List, Tuple

from collections import Counter

# Import & explore dataset

In [2]:
aylien = pd.read_pickle('./datasets/Aylien/Aylien_business_articles_10k.pickle')

In [3]:
# rename id col.
aylien.rename(columns={'id': 'article_id'}, inplace=True)

In [4]:
print(len(aylien))
aylien.info()

10000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype                  
---  ------        --------------  -----                  
 0   article_id    10000 non-null  int64                  
 1   title         10000 non-null  object                 
 2   published_at  10000 non-null  datetime64[ns, tzutc()]
 3   source        10000 non-null  object                 
 4   body          10000 non-null  object                 
dtypes: datetime64[ns, tzutc()](1), int64(1), object(3)
memory usage: 468.8+ KB


In [5]:
aylien.head(3)

,article_id,title,published_at,source,body
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...


## Reduced subset:

In [6]:
aylien_5k = aylien[:5000].copy()

print(len(aylien_5k))
aylien_5k.head()

5000


,article_id,title,published_at,source,body
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,2023-06-02 12:46:28+00:00,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala..."
4,5709271283,Another Soft Market Is Inevitable,2023-06-02 12:46:23+00:00,Carrier Management,New You can now listen to Carrier Management a...


# Prompt generation

## Investigating potential prompts

1. "give me an answer only in cypher language that i can paste into neo4j to create a graph. What is the single main event and relation described by the article regarding IIC Padmalochan Manigarahi?  The article is: Mayurbhanj: One person was killed, while two others sustained critical injuries when they were brutally attacked by a youth on suspicion of practising black magic on Sunday night.\nThe incident has been reported from Kainal village in Jualibhanga panchayat under Morada police station limits in Odisha's Mayurbhanj district.\n  The deceased has been identified as Gauri Tudu (70) and the injured as Kuni Tudu (42) and Kalicharan Tudu (47).\n  According to reports, a youth named Durga Charan Tudu suddenly attacked the three while they were sitting in their home last night. Durga Charan Tudu allegedly attacked the three with a sharp weapon, which left septuagenarian Gauri Tudu dead on the spot, while two members of the family were severely injured and dropped unconscious. He bore a grudge against the Tudu family suspecting that they practised witchcraft and were responsible for several ills in the village.\n  Soon after the gruesome incident, the three were rushed to the nearby community health center where doctors declared Gauri Tudu brought dead. Kuni Tudu and Kalicharan Tudu were shifted to the PRM Hospital in Baripada as their condition deteriorated. Currently, Kuni Tudu and Kalicharan Tudu are undergoing treatment in the hospital and their condition is said to be critical.\n  On being informed, Morada police IIC Padmalochan Manigarahi and Chitrada outpost officer Niranjan Saren reached the spot with their teams. They sent Gauri's body for post-mortem and detained Durga Charan Tudu for interrogation. An investigation into the incident is underway."

* Notes on above:
    * only sometimes produces cypher query.
    * and if it does, difficult to get in a clean plaintext format that can be piped out.
    * cypher query often not accurate, if produced.
    * response from CGPT too large/cumbersome. defeats purpose.
    

In [7]:

# Create a function to generate the prompt based on the row values
def generate_prompt(row):

    body = row['body']
    
    prompt = f'''For the main organisation discussed in this article (if any), give me information as follows and nothing else. If there is no organiastion discussed, respond with "NA".
    ###
    Desired response format:
    ###
    org: <main organisation discussed>
    country: <country of main organisation>
    state: <state/province of main organisation>
    city: <city/town of main organisation>
    industry: <industry of the organisation>
    risks: <semi-colon-separated list of risks faced by main organisation> Summarise each risk in a phrase of up to 3 words.
    items_sold: <semicolon-separated list of items sold by main organisation>
    service_provided: <semi-colon-separated list of services provided by organisation>
    business_relations: <semi-colon-separated list of organisations with a business relation to the main organisation>
    ###
    The provided article is:
    ###
    {body}
    ###'''
    
    return prompt

# Add a new 'prompt' column by applying the generate_prompt function to each row
aylien_5k['prompt'] = aylien_5k.apply(generate_prompt, axis=1)
aylien_5k['prompt'] = aylien_5k['prompt'].astype(str)


In [8]:
aylien_5k.head()

,article_id,title,published_at,source,body,prompt
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,For the main organisation discussed in this ar...
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,For the main organisation discussed in this ar...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...,For the main organisation discussed in this ar...
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,2023-06-02 12:46:28+00:00,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala...",For the main organisation discussed in this ar...
4,5709271283,Another Soft Market Is Inevitable,2023-06-02 12:46:23+00:00,Carrier Management,New You can now listen to Carrier Management a...,For the main organisation discussed in this ar...


## Export df with prompts to serve as input reference

In [9]:
# Export DataFrame to pickle file
pickle_file_path = './datasets/Aylien/zero_shot_aylien_business_5k_with_prompts.pickle'
aylien_5k.to_pickle(pickle_file_path)

In [10]:
# convert published_at to be timezone-unaware, to export to excel

aylien_5k['published_at'] = pd.to_datetime(aylien_5k['published_at']).dt.tz_localize(None)

# Convert 'published_at' column to 'DD-Month-YYYY' format
aylien_5k['published_at'] = aylien_5k['published_at'].dt.strftime('%d-%B-%Y')

In [11]:
# Export DataFrame to Excel file
excel_file_path = './datasets/Aylien/zero_shot_aylien_business_5k_with_prompts.xlsx'
aylien_5k.to_excel(excel_file_path, index=False)

### Convert to prompts to all_prompts list to explore.

In [12]:
all_prompts = aylien_5k['prompt'].tolist()

In [13]:
# observe single prompt as example.
all_prompts[0]

'For the main organisation discussed in this article (if any), give me information as follows and nothing else. If there is no organiastion discussed, respond with "NA".\n    ###\n    Desired response format:\n    ###\n    org: <main organisation discussed>\n    country: <country of main organisation>\n    state: <state/province of main organisation>\n    city: <city/town of main organisation>\n    industry: <industry of the organisation>\n    risks: <semi-colon-separated list of risks faced by main organisation> Summarise each risk in a phrase of up to 3 words.\n    items_sold: <semicolon-separated list of items sold by main organisation>\n    service_provided: <semi-colon-separated list of services provided by organisation>\n    business_relations: <semi-colon-separated list of organisations with a business relation to the main organisation>\n    ###\n    The provided article is:\n    ###\n    Nike  96.41 CHF -1.99%  Charts  News  Analysen  Kaufen  Verkaufen Shares of Nike Inc.\nNKE 

# Exploring prompts

## Investigate average length of tokens

* Important for pricing forecast and LLM API restrictions
* must use tiktoken (and not nltk), with the correct encoder.

### with tiktoken

In [14]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
#encoding = tiktoken.get_encoding("cl100k_base")

In [15]:
# fns to count strings in list of prompts:

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def count_tokens_in_list(prompt_list: list, encoding_name: str) -> list:
    """Returns a list of integers representing the number of tokens in each string in the input list."""
    token_counts = []
    for prompt in prompt_list:
        num_tokens = num_tokens_from_string(prompt, encoding_name)
        token_counts.append(num_tokens)
    return token_counts


In [16]:
encoding_name = "cl100k_base" # used for gpt-3.5-turbo
token_counts = count_tokens_in_list(all_prompts, encoding_name)

In [17]:
min_tokens = min(token_counts)
max_tokens = max(token_counts)
total_tokens = sum(i for i in token_counts if isinstance(i, int))
average_tokens = total_tokens / len(all_prompts)

print(f"Number of tokens on the smallest prompt: {min_tokens}")
print(f"Number of tokens on the largest prompt: {max_tokens}")
print(f"Total number of tokens for all prompts: {total_tokens}")
print(f"Average number of tokens in all_prompts: {average_tokens}")

Number of tokens on the smallest prompt: 195
Number of tokens on the largest prompt: 14303
Total number of tokens for all prompts: 4402221
Average number of tokens in all_prompts: 880.4442


#### Cross-check above count against count in 'prompt' col of df.

In [18]:

# Apply the num_tokens_from_string function to each row in the 'prompt' column
aylien_5k['token_count'] = aylien_5k['prompt'].apply(lambda x: num_tokens_from_string(x, "cl100k_base"))

# Get summary statistics
min_tokens = aylien_5k['token_count'].min()
max_tokens = aylien_5k['token_count'].max()
total_tokens = aylien_5k['token_count'].sum()
average_tokens = total_tokens / len(aylien_5k)

print(f"Number of tokens on the smallest prompt: {min_tokens}")
print(f"Number of tokens on the largest prompt: {max_tokens}")
print(f"Total number of tokens for all prompts: {total_tokens}")
print(f"Average number of tokens in all prompts: {average_tokens}")

Number of tokens on the smallest prompt: 195
Number of tokens on the largest prompt: 14303
Total number of tokens for all prompts: 4402221
Average number of tokens in all prompts: 880.4442


### Estimating max response tokens if prompt works correctly (update to reflect prompt used)

In [19]:
#num_tokens_from_string("facing risk: yes. type of risk: thisis some text for a risk type.", "cl100k_base")

num_tokens_from_string(
    
    '''
    "You will be extracting information from the provided article in the specified format."
    org: Silicon Valley Bank
    country: U.S.
    state: California
    city: Santa Clara
    industry: tech erogih eroigh erogih ergohi ergohi
    risks: political; financial; mismanagement; regulatory, eorigherogi, oeirhgoerihg , eorigheriogh
    items_sold: NA, peiorgherhg, eoirgh eroighj er, oierhg, eioprj, eighjowh, rijgio, oeirgjerj ,eoirgj
    service_provided: tech-lending; ergoiperg, eorigheropigh, oeighioerhg, oergheroig
    business_relations: Credit Suisse, eoprgje, oeirhgerhgho, woiejh, wioehr, fjkr we  d f f, wejfj r 
    '''
                       ,"cl100k_base" )

216

### Truncate tokens of long prompts

* gpt-3.5-turbo has max tokens of 4,096 tokens
* This includes prompt and response tokens combined.
* response tokens should be short due to the attempt at prompt restrictions;
    * i.e. Provide answers only in the format of <facing risk: <'yes'/'no'>. type of risk: < risk type >.> and nothing else.
* so a generous estimate of response tokens would be 400, providing gpt-3.5-turbo successfully adheres to above prompting.
* Therefore truncate prompt tokens to 3500 to be safe (truncating from the end backwards).

In [20]:
def truncate_prompt(prompt: str, encoding_name: str, max_tokens: int) -> str:
    """Truncates a text string to the specified number of tokens."""
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(prompt)[:max_tokens]
    return encoding.decode(tokens)


def count_tokens_for_truncating(prompt_list: list, encoding_name: str, max_tokens: int) -> list:
    """Returns a list of strings with a maximum of max_tokens tokens."""
    token_counts = []
    truncated_prompts = []
    for prompt in prompt_list:
        num_tokens = num_tokens_from_string(prompt, encoding_name)
        if num_tokens > max_tokens:
            truncated_prompt = truncate_prompt(prompt, encoding_name, max_tokens)
            token_counts.append(max_tokens)
        else:
            truncated_prompt = prompt
            token_counts.append(num_tokens)
        truncated_prompts.append(truncated_prompt)
    return truncated_prompts, token_counts


In [21]:
encoding_name = "cl100k_base" # used for gpt-3.5-turbo
max_tokens = 3500 # Update this based on model to be used in 'Generating responses' section to correspond to token limitations.

truncated_prompts, token_counts = count_tokens_for_truncating(all_prompts, encoding_name, max_tokens)

In [22]:
# Compare output to previous token counts.

token_counts_truncated_prompts = count_tokens_in_list(truncated_prompts, encoding_name)
min_tokens_truncated_prompts = min(token_counts_truncated_prompts)
max_tokens_truncated_prompts = max(token_counts_truncated_prompts)
total_tokens_truncated_prompts = sum(i for i in token_counts_truncated_prompts if isinstance(i, int))
average_tokens_truncated_prompts = total_tokens_truncated_prompts / len(truncated_prompts)

print(f"Number of tokens on the smallest prompt: {min_tokens_truncated_prompts}")
print(f"Number of tokens on the largest prompt: {max_tokens_truncated_prompts}")
print(f"Total number of tokens for all prompts: {total_tokens_truncated_prompts}")
print(f"Average number of tokens in all_prompts: {average_tokens_truncated_prompts}")

# Compare output to previous output (prior to truncation).

Number of tokens on the smallest prompt: 195
Number of tokens on the largest prompt: 3500
Total number of tokens for all prompts: 4306044
Average number of tokens in all_prompts: 861.2088


### Above truncation was applied to the all_prompts list of strings.
* Need to now apply to the aylien_5k['prompt'] col

In [23]:
aylien_5k.head(2)

,article_id,title,published_at,source,body,prompt,token_count
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,02-June-2023,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,For the main organisation discussed in this ar...,491
1,5709292390,Renault Customers Planning Criminal Complaint ...,02-June-2023,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,For the main organisation discussed in this ar...,592


In [24]:
def num_tokens_from_string(text: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(text))

def truncate_prompt(prompt: str, encoding_name: str, max_tokens: int) -> str:
    """Truncates a text string to the specified number of tokens."""
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(prompt)[:max_tokens]
    return encoding.decode(tokens)

def truncate_prompts_in_df(df: pd.DataFrame, encoding_name: str, max_tokens: int):
    """Truncates prompts in the DataFrame if their token count exceeds the max limit."""
    df.loc[df['token_count'] > max_tokens, 'prompt'] = df.loc[df['token_count'] > max_tokens, 'prompt'].apply(
        lambda x: truncate_prompt(x, encoding_name, max_tokens)
    )
    df.loc[df['token_count'] > max_tokens, 'token_count'] = max_tokens  # Update token_count

# Apply truncation
truncate_prompts_in_df(aylien_5k, "cl100k_base", 3500)

aylien_5k.head()

,article_id,title,published_at,source,body,prompt,token_count
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,02-June-2023,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,For the main organisation discussed in this ar...,491
1,5709292390,Renault Customers Planning Criminal Complaint ...,02-June-2023,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,For the main organisation discussed in this ar...,592
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,02-June-2023,Newsdaemon.com,Search Search Search Search Sana Meer Ju...,For the main organisation discussed in this ar...,1479
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,02-June-2023,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala...",For the main organisation discussed in this ar...,703
4,5709271283,Another Soft Market Is Inevitable,02-June-2023,Carrier Management,New You can now listen to Carrier Management a...,For the main organisation discussed in this ar...,1851


In [25]:
# Get truncated summary statistics:

min_tokens = aylien_5k['token_count'].min()
max_tokens = aylien_5k['token_count'].max()
total_tokens = aylien_5k['token_count'].sum()
average_tokens = total_tokens / len(aylien_5k)

print(f"Number of tokens on the smallest prompt: {min_tokens}")
print(f"Number of tokens on the largest prompt: {max_tokens}")
print(f"Total number of tokens for all prompts: {total_tokens}")
print(f"Average number of tokens in all prompts: {average_tokens}")
print("Number of prompts: ", len(aylien_5k))

Number of tokens on the smallest prompt: 195
Number of tokens on the largest prompt: 3500
Total number of tokens for all prompts: 4306044
Average number of tokens in all prompts: 861.2088
Number of prompts:  5000


# Prompt structure

In [26]:
# define / restrict input size:
aylien_5k_input = aylien_5k.head(1000).copy()

In [28]:
print(len(aylien_5k_input))

1000


In [29]:
openai.api_key = os.getenv('OPENAI_KEY')

In [30]:
# model:
GPT_MODEL = "gpt-3.5-turbo"

# Retry parameters
max_retries = 3
retry_delay = 5  # in seconds

# Initialize a new column 'responses' in the dataframe
aylien_5k_input.loc[:, 'responses'] = ''

def generate_responses(input_df):
    for idx, row in tqdm(input_df.iterrows(), total=len(input_df), desc="Generating responses"):
        try:
            query = row['prompt']

            if query is None or query.strip() == '':
                print(f"Empty prompt at index {idx}. Skipping this row.")
                continue

            retries = 0
            while retries < max_retries:
                try:
                    response = openai.ChatCompletion.create(
                        model=GPT_MODEL,
                        messages=[
                            {"role": "system", "content": "You will be extracting information from the provided article in the specified format."},
                            {"role": "user", "content": query}, # In general, gpt-3.5-turbo-0301 does not pay strong attention to the system message, and therefore important instructions are often better placed in a user message.
                        ],
                        temperature=0,
                        max_tokens=400,  # max tokens in response. token limit (4096) must be < largest query + message content + max_tokens
                    )

                    response_content = response['choices'][0]['message']['content']
                    print(response_content)  # Print just the message content

                    # Add the response to the 'responses' column of the dataframe
                    input_df.loc[idx, 'responses'] = response_content

                    break  # Break out of the retry loop if successful

                except Exception as e:
                    print(f"Error generating response for prompt at index {idx}: {str(e)}")
                    print(f"Query: {query}")
                    retries += 1
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)

            if retries == max_retries:
                print(f"Max retries reached for prompt at index {idx}. Skipping this row.")

            # Add a delay between requests to avoid overwhelming the API
            time.sleep(random.uniform(0.5, 1.5))

        except Exception as e:
            print(f"Error generating response for prompt at index {idx}: {str(e)}")
            print(f"Query: {query}")

## Run prompts

In [31]:
generate_responses(aylien_5k_input)

Generating responses:   0%|          | 0/1000 [00:00<?, ?it/s]

org: Nike Inc.
country: NA
state: NA
city: NA
industry: Athletic apparel
risks: Stale footwear
items_sold: NA
service_provided: NA
business_relations: Macy's Inc.


Generating responses:   0%|          | 1/1000 [00:04<1:11:50,  4.32s/it]

org: RENAULT
country: NA
state: NA
city: NA
industry: Automotive
risks: Supply chain; Economic downturn; Competition
items_sold: Cars; Commercial vehicles
service_provided: NA
business_relations: NA


Generating responses:   0%|          | 2/1000 [00:08<1:13:20,  4.41s/it]

...

Generating responses: 100%|█████████▉| 998/1000 [1:05:44<00:07,  3.84s/it]

org: Code and Theory
country: NA
state: NA
city: NA
industry: Design and Digital Transformation
risks: Status Quo Bias; Risk Avoidance; Hyperbolic Discounting
items_sold: NA
service_provided: Design Transformation services
business_relations: McKinsey, large global investment management firm, large financial institution, auditing firm, one of the largest manufacturers of PCs and mobile devices


Generating responses: 100%|█████████▉| 999/1000 [1:05:50<00:04,  4.39s/it]

org: Toyota
country: NA
state: NA
city: NA
industry: Automotive
risks: Data leak; Cybersecurity threats
items_sold: Cars and automotive products
service_provided: Automotive services
business_relations: NA


Generating responses: 100%|██████████| 1000/1000 [1:05:53<00:00,  3.95s/it]


In [32]:
print(len(aylien_5k))
aylien_5k_input.head()

5000


,article_id,title,published_at,source,body,prompt,token_count,responses
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,02-June-2023,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,For the main organisation discussed in this ar...,491,org: Nike Inc.\ncountry: NA\nstate: NA\ncity: ...
1,5709292390,Renault Customers Planning Criminal Complaint ...,02-June-2023,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,For the main organisation discussed in this ar...,592,org: RENAULT\ncountry: NA\nstate: NA\ncity: NA...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,02-June-2023,Newsdaemon.com,Search Search Search Search Sana Meer Ju...,For the main organisation discussed in this ar...,1479,org: Mersana Therapeutics Inc.\ncountry: NA\ns...
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,02-June-2023,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala...",For the main organisation discussed in this ar...,703,org: Galaxy Digital\ncountry: NA\nstate: NA\nc...
4,5709271283,Another Soft Market Is Inevitable,02-June-2023,Carrier Management,New You can now listen to Carrier Management a...,For the main organisation discussed in this ar...,1851,org: RLI Corp.\ncountry: NA\nstate: NA\ncity: ...


## Post-processing & exporting

In [33]:
data = []

def process_response(response):
    # Remove < and > characters and trailing /
    response = response.replace('<', '').replace('>', '').replace('#', '')

    item = {}
    lines = response.split('\n')
    for line in lines:
        if ': ' in line:
            key, value = line.split(': ', 1)
            key = key.lower()  # Convert key to lowercase
            if value.strip() in ["NA", "NA."]:
                item[key] = ""
            else:
                item[key] = value.strip()
    data.append(item)
    return item

# Apply the processing function to the 'responses' column and assign the result to a new column 'responses_PP'
aylien_5k_input['responses_PP'] = aylien_5k_input['responses'].apply(process_response)

# Save 'responses_PP' column as JSON
output_file = "./results/zero_shot_aylien_business_5k_responses.json"

# Write the 'responses_PP' column data to the JSON file
with open(output_file, "w") as file:
    json.dump(aylien_5k_input['responses_PP'].tolist(), file, indent=2)

print(f"Populated schemas saved to {output_file}")

Populated schemas saved to ./results/zero_shot_aylien_business_5k_responses.json


## Export df with responses

In [34]:
aylien_5k_input.head(2)

,article_id,title,published_at,source,body,prompt,token_count,responses,responses_PP
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,02-June-2023,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,For the main organisation discussed in this ar...,491,org: Nike Inc.\ncountry: NA\nstate: NA\ncity: ...,"{'org': 'Nike Inc.', 'country': '', 'state': '..."
1,5709292390,Renault Customers Planning Criminal Complaint ...,02-June-2023,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,For the main organisation discussed in this ar...,592,org: RENAULT\ncountry: NA\nstate: NA\ncity: NA...,"{'org': 'RENAULT', 'country': '', 'state': '',..."


In [35]:
# export to pickle in case of future use.
aylien_5k_input.to_pickle("./results/zero_shot_aylien_business_5k_responses.pkl")

# export to excel for manual verification:
aylien_5k_input.to_excel("./results/zero_shot_aylien_business_5k_responses.xlsx")